In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool
def web_search(query: str) -> Dict[str, Any]:
    """
    Search the web for information
    """
    return tavily_client.search(query)


@tool
def sql_query(query: str) -> str:
    """
    Obtain information from the database using SQL queries
    """

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [3]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [4]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """
    Dynamically call tools based on the runtime context
    """

    user_role = request.runtime.context.user_role

    if user_role == "internal":
        pass # Internal users get access to all tools
    else:
        tools = [web_search] # External users only get access to web search
        request = request.override(tools=tools)
    
    return handler(request)

In [5]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [7]:
from langchain.messages import HumanMessage

user_query = HumanMessage(content="How many artists are in the database?")

response = agent.invoke(
    {"messages": [user_query]},
    context={
        "user_role": "internal"
    }
)

print(response["messages"][-1].content)

275 artists.

Note: The relevant table is named Artist (not artists). If you want, I can pull the first few artist names or compute statistics like total tracks per artist.


https://smith.langchain.com/public/7a51ffa9-3066-4b8d-a0c2-34357906ba28/r

In [8]:
user_query = HumanMessage(content="How many artists are in the database?")

response = agent.invoke(
    {"messages": [user_query]},
    context={
        "user_role": "external"
    }
)

print(response["messages"][-1].content)

I don’t have enough context to know which database you mean. Can you provide:

- The database name and type (SQL like PostgreSQL/MySQL, or NoSQL like MongoDB, or a REST API)?
- The table/collection name (e.g., artists)?
- Whether you want an exact count or an approximate count?
- If you can share the schema or a connection method (query you can run, or API endpoint)?

If you want, I can give you the exact query templates:
- SQL: SELECT COUNT(*) AS artist_count FROM artists;
- MongoDB: db.artists.countDocuments({});
- REST API: GET /artists/count or GET /artists?limit=1 and check the total in the response’s metadata.

Provide the details and I’ll compute the count or guide you step-by-step.


## Dynamic Tools with improved method of providing DB Schema as Context to Agent

In [2]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.messages import HumanMessage, SystemMessage
from langchain_community.utilities import SQLDatabase
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from dataclasses import dataclass
from tavily import TavilyClient
from typing import Dict, Any
from typing import Callable
from pprint import pprint

DB_URI = "sqlite:///resources/Chinook.db"

tavily_client = TavilyClient()

#           Both of these Data Classes combined into one class below
# @dataclass
# class SQLContext:
#     db: SQLDatabase
#     schema: str # one-time snapshot

# @dataclass
# class UserRole:
#     user_role: str = "external"

@dataclass
class AppContext:
    db: SQLDatabase
    schema: str # one-time snapshot
    user_role: str = "external"

def build_sql_context(db_uri: str) -> AppContext:
    db = SQLDatabase.from_uri(db_uri)
    schema = db.get_table_info() # one-time schema capture
    return AppContext(db=db, schema=schema)

ctx = build_sql_context(DB_URI)

In [3]:
@tool
def web_search(query: str) -> Dict[str, Any]:
    """
    Search the web for information
    """
    return tavily_client.search(query)


@tool
def get_db_schema(runtime: ToolRuntime) -> str:
    """
    Get the database schema (one-time snapshot)
    """
    return runtime.context.schema


@tool
def sql_query(runtime: ToolRuntime, query: str) -> str:
    """
    Obtain information from the database using SQL queries
    """

    try:
        return runtime.context.db.run(query)
    except Exception as e:
        return f"Error: {e}"


@wrap_model_call
def dynamic_tool_call(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """
    Dynamically call tools based on the runtime context
    """

    user_role = request.runtime.context.user_role

    if user_role == "internal":
        print(">> Allowing all tools <<")
        pass # Internal users get access to all tools
    else:
        print(">> Only allowing the web search tool <<")
        tools = [web_search] # External users only get access to web search
        request = request.override(tools=tools)
    
    return handler(request)

In [4]:
system_prompt = SystemMessage(
    content=(
        """
        You are an expert AI assistant and you have some tools available to you.

        Depending on your role, you may have a SQL query tool available. 
        If your role allows and if you are required to do a SQL query, **always** call `get_db_schema` and only call it **once**. 
        Reuse the schema as needed before writing SQL, and use only tables/columns that appear there. 
        Once you have the schema, go ahead and craft a SQL query to query the database. Attempt the query as many times
        as required until you have the data.

        Do not ask follow up questions to the user, just use your tools and answer the users question.
        """
    )
)

agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search, get_db_schema, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=AppContext,
    system_prompt=system_prompt
)

In [5]:
agent.context_schema

__main__.AppContext

In [6]:
user_query = HumanMessage(
    content="Who is the most popular artist beginning with 'S'?"
)

response = agent.invoke(
    {"messages": [user_query]},
    context={
            "user_role": "internal",
            "db": ctx.db,
            "schema": ctx.schema
    }
)

print(response["messages"][-1].content)

>> Allowing all tools <<


/Users/johnmachado/Desktop/LangChain 1.0 Foundations/env/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=AppContext(db=<langchain_...', user_role='internal'), input_type=AppContext])
  return self.__pydantic_serializer__.to_python(


>> Allowing all tools <<


/Users/johnmachado/Desktop/LangChain 1.0 Foundations/env/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=AppContext(db=<langchain_...', user_role='internal'), input_type=AppContext])
  return self.__pydantic_serializer__.to_python(


>> Allowing all tools <<
Smashing Pumpkins — Revenue: 23.76 (currency units).


https://smith.langchain.com/public/66a067fd-007d-4e03-9d3c-990d4989ea6d/r

In [7]:
user_query = HumanMessage(
    content="Who is the most popular music artist beginning with 'S'?"
)

response = agent.invoke(
    {"messages": [user_query]},
    context={
            "user_role": "external",
            "db": ctx.db,
            "schema": ctx.schema
    }
)

print(response["messages"][-1].content)

>> Only allowing the web search tool <<
>> Only allowing the web search tool <<
>> Only allowing the web search tool <<
Shawn Mendes.

Among artists whose name starts with "S", he has the highest Spotify monthly listeners, at about 47,491,979 monthly listeners (Rank 73 on ChartMasters' Most Monthly Listeners on Spotify list). Source: ChartMasters.


https://smith.langchain.com/public/ae18a5aa-c09e-445a-a076-cae30d22a9d2/r